In [4]:
import optuna

In [5]:
import xgboost

In [6]:
!pip list

Package                           Version
--------------------------------- ------------------
absl-py                           2.2.0
aext-assistant                    4.1.0
aext-assistant-server             4.1.0
aext-core                         4.1.0
aext-core-server                  4.1.0
aext-panels                       4.1.0
aext-panels-server                4.1.0
aext-project-filebrowser-server   4.1.0
aext-share-notebook               4.1.0
aext-share-notebook-server        4.1.0
aext-shared                       4.1.0
aext-toolbox                      4.1.0
aiobotocore                       2.12.3
aiohappyeyeballs                  2.4.0
aiohttp                           3.10.5
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
alembic                           1.13.3
altair                            5.0.1
anaconda-anon-usage               0.4.4
anaconda-catalogs                 0.2.0
anaconda-cli-base    

In [7]:
!pip install optuna

In [8]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [9]:
df=pd.read_csv("Admission_Prediction.csv")

In [10]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [12]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [13]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [14]:
df["GRE Score"]=df["GRE Score"].fillna(df["GRE Score"].median())
df["TOEFL Score"]=df["TOEFL Score"].fillna(df["TOEFL Score"].median())
df["University Rating"]=df["University Rating"].fillna(df["University Rating"].median())

In [15]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [16]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [17]:
X=df.drop(["Serial No.","Chance of Admit"],axis=1)

In [18]:
y=df["Chance of Admit"]

In [19]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [20]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=25)

In [21]:
X_train.shape

(375, 7)

In [22]:
X_test.shape

(125, 7)

In [23]:
y_train.shape

(375,)

In [24]:
y_test.shape

(125,)

In [25]:
std_sca=StandardScaler()

In [26]:
X_train=std_sca.fit_transform(X_train)

In [27]:
X_train.shape

(375, 7)

In [28]:
X_test=std_sca.transform(X_test)

In [29]:
X_test.shape

(125, 7)

In [30]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [31]:
print(X)
print(y)

     GRE Score  TOEFL Score  University Rating  SOP  LOR  CGPA  Research
0        337.0        118.0                4.0  4.5  4.5  9.65         1
1        324.0        107.0                4.0  4.0  4.5  8.87         1
2        317.0        104.0                3.0  3.0  3.5  8.00         1
3        322.0        110.0                3.0  3.5  2.5  8.67         1
4        314.0        103.0                2.0  2.0  3.0  8.21         0
..         ...          ...                ...  ...  ...   ...       ...
495      332.0        108.0                5.0  4.5  4.0  9.02         1
496      337.0        117.0                5.0  5.0  5.0  9.87         1
497      330.0        120.0                5.0  4.5  5.0  9.56         1
498      312.0        103.0                4.0  4.0  5.0  8.43         0
499      327.0        113.0                4.0  4.5  4.5  9.04         0

[500 rows x 7 columns]
0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497 

In [40]:
def objective(trial, data=X, target=y):
    # Split the data into training and testing sets
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25, random_state=2)
    
    # Define the hyperparameter search space
    param = {
        'tree_method': 'auto',
        'lambda': trial.suggest_loguniform('lambda', 1e-4, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-4, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.0001, 0.001, 0.01, 0.02, 0.03, 0.1]),
        'n_estimators': 300,
        'max_depth': trial.suggest_categorical('max_depth', [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
        'random_state': trial.suggest_categorical('random_state', [10, 20, 30, 2000, 3454, 243123]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 200)
    }
    
    # Initialize the model with the suggested hyperparameters
    model = xgb.XGBRegressor(**param)
    
    # Train the model
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)], verbose=False)
    
    # Predict on the test set
    pred = model.predict(test_x)
    
    # Calculate the mean squared error
    mse = mean_squared_error(test_y, pred)
    return mse

In [42]:
# Create a study object with the direction specified
find_params = optuna.create_study(direction="minimize")

# Optimize the objective function
find_params.optimize(objective, n_trials=50)

# Retrieve the best hyperparameters
find_params.best_trial.params

[I 2025-03-22 22:10:07,698] A new study created in memory with name: no-name-da51843b-0fc8-4e11-bf61-e7563787cdd0
C:\Users\vaish\AppData\Local\Temp\ipykernel_6424\425629765.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-4, 10.0),
C:\Users\vaish\AppData\Local\Temp\ipykernel_6424\425629765.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-4, 10.0),
[I 2025-03-22 22:10:08,002] Trial 0 finished with value: 0.01945461989677314 and parameters: {'lambda': 2.682374803487207, 'alpha': 2.2745952306251795, 'colsample_bytree': 0.9, 'subsample': 0.3, 'learning_rate': 0.03, 'max_depth

{'lambda': 1.8688964859913604,
 'alpha': 0.00018476188251752935,
 'colsample_bytree': 0.5,
 'subsample': 0.6,
 'learning_rate': 0.02,
 'max_depth': 4,
 'random_state': 243123,
 'min_child_weight': 12}

In [44]:
best_params={'lambda': 1.1038299415263448,
 'alpha': 0.2002246174362676,
 'colsample_bytree': 0.3,
 'subsample': 0.9,
 'learning_rate': 0.008,
 'max_depth': 4,
 'random_state': 3454,
 'min_child_weight': 30}

In [46]:
!python --version

Python 3.12.7


In [48]:
!pip show optuna

Name: optuna
Version: 4.2.1
Summary: A hyperparameter optimization framework
Home-page: https://optuna.org/
Author: Takuya Akiba
Author-email: 
License: 
Location: C:\Users\vaish\anaconda3\Lib\site-packages
Requires: alembic, colorlog, numpy, packaging, PyYAML, sqlalchemy, tqdm
Required-by: 


In [50]:
!python --version

Python 3.12.7


In [52]:
model=xgb.XGBRegressor(**best_params)

In [54]:
model=xgb.XGBRegressor(**best_params)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
#r2score=[0,1]
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)

0.8208664807099978

In [56]:
y_pred=model.predict(X_test)

In [58]:
#r2score=[0,1]
from sklearn.metrics import r2_score

r2_score(y_test,y_pred)

0.5547875701898823

In [60]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)

0.8174036457478338

In [62]:
import os
import logging

# Specify the directory and file
dir_path = r'/content/logs/'
log_file = 'system.txt'
full_path = os.path.join(dir_path, log_file)
print(full_path)

/content/logs/system.txt


In [64]:
# Check if the directory exists and create it if necessary
os.makedirs(dir_path, exist_ok=True)

In [66]:
# Set up logging
# Get a logger instance (this will fetch the root logger)
logger = logging.getLogger()

In [68]:
# Set the level of the logger to CRITICAL
# This means it will handle events of level CRITICAL and above
logger.setLevel(logging.CRITICAL)

In [70]:
# Create a FileHandler instance to write logs to a file
handler = logging.FileHandler(full_path)

In [72]:

# Add the handler to the logger
# This connects the logger to the handler so that logs get written to the file
logger.addHandler(handler)


In [74]:
def LetUsCheckSystem(sys):
    if sys != 'OK':
        logging.critical('System failure: %s', sys)

LetUsCheckSystem('You need to handle the issue now')
handler.close()

In [76]:
import os
import logging

# Specify the directory and file
dir_path = r'/content/logs/'
log_file = 'system.txt'
full_path = os.path.join(dir_path, log_file)

# Check if the directory exists and create it if necessary
os.makedirs(dir_path, exist_ok=True)

# Set up logging
# Get a logger instance (this will fetch the root logger)
logger = logging.getLogger()

# Set the level of the logger to CRITICAL
# This means it will handle events of level CRITICAL and above
logger.setLevel(logging.CRITICAL)

# Create a FileHandler instance to write logs to a file
handler = logging.FileHandler(full_path)

# Set the format of the logs using a Formatter
# This format includes the log timestamp, log level and log message
handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s:%(message)s'))

# Add the handler to the logger
# This connects the logger to the handler so that logs get written to the file
logger.addHandler(handler)




In [78]:
def LetUsCheckSystem(sys):
    if sys != 'OK':
        logging.critical('System failure: %s', sys)

LetUsCheckSystem('You need to handle the issue now')
handler.close()